**CLASSIFIER DEFINITIONS AND INITILISATIONS**

In [ ]:
# numpy provides arrays and useful functions for working with them
import numpy
# suppressing scientific notation in numpy so that numbers are mostly displayed in decimal format
numpy.set_printoptions(suppress=True)

# scipy.special for the sigmoid function expit()
import scipy.special
# scipy.ndimage for rotating image arrays
import scipy.ndimage

In [ ]:
# softmax function for final layer (optimal for cross-entropy loss)
def softmax(x):
    exp_x = numpy.exp(x - numpy.max(x, axis=1, keepdims=True))
    return exp_x / numpy.sum(exp_x, axis=1, keepdims=True)

In [ ]:
# ReLU function for hidden layers
def relu(x):
    return numpy.where(x > 0, x, 0.0)

# ReLU derivative for backprop
def drelu(x):
    return numpy.where(x > 0, 1.0, 0.0)

In [ ]:
# sigmoid function as an alternative option for hidden layers
def sigmoid(x):
    return scipy.special.expit(x)

# sigmoid derivative for backprop
def dsigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [ ]:
# general derivative function
# works for both ReLU and sigmoid, for ease of switching between the two in backpropagation
# also includes softmax to enable optimisation for output nodes

# takes in a function and an array and returns the derivative of that function applied to the array
def deriv(fn, x):
    if fn == relu:
        return numpy.where(x > 0, 1.0, 0.0)
    elif fn == softmax:
        J0 = softmax(x).reshape(-1, 1)
        jacobian = numpy.diagflat(J0) - numpy.dot(J0, J0.T)
        return jacobian
    elif fn == sigmoid:
        return scipy.special.expit(x) * (1 - scipy.special.expit(x))

In [ ]:
# DNN (Deep Neural Network) class for MNIST classifier
class neuralNetwork:

    # all self parameters are initialised / declared in initialisation of class
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        
        # compiling a list of all layer node numbers for indexing later
        self.nodes = [inputnodes] + hiddennodes + [outputnodes]

        self.weights = []
        self.biases = []

        # iterating over layers to create arrays of weights and biases with the right dimensions
        # DNN class will automatically adapt to any number of hidden layers and nodes with this loop structure
        # NB: usage of for loops does slow down training slightly, but only by about 5%
        for i in range(len(self.nodes)-1):
            
            # using Kaiming intilisation for the weights (optimal for ReLU), i.e. normal distribution with mean 0 and stdev sqrt(2/n)
            self.weights.append(numpy.random.normal(0.0, numpy.sqrt(2/self.nodes[i]), (self.nodes[i], self.nodes[i+1])))
            
            # zero vectors for biases corresponding to each layer except inputs
            self.biases.append(numpy.zeros(self.nodes[i+1],))

        # variables for loss monitoring
        # tracker will be updated at the start of each training batch, using the current loss (with parameters from the last DNN update)
        self.loss_tracker = 0
        # alpha is the degree of smoothing for the exponential moving average (defined later)
        self.alpha = 0.9999

        self.lr = learningrate
        
        # separate activation functions for hidden and output layers
        # usage of ReLU achieves best performance, but output should be probabilities summing to 1, which softmax does
        self.hidden_activation = lambda x: relu(x)
        self.output_activation = lambda x: softmax(x)

        # specifying the activation functions for general deriv function
        self.hidden_fn = relu
        self.output_fn = softmax
        
        pass
    
    def train(self, inputs_list, targets_list):
        
        # ensuring inputs and targets for training are of the right dimension and shape (corresponding to input and final layers)
        inputs = numpy.reshape(numpy.array(inputs_list, ndmin=2), [batch_size, self.nodes[0]])
        targets = numpy.reshape(numpy.array(targets_list, ndmin=2), [batch_size, self.nodes[-1]])

        # lists for inputs to each layer and activation function outputs
        # image values are both the input and output for layer 0
        layer_inputs = [inputs]
        layer_outputs = [inputs]

        # computing outputs and inputs for hidden layers
        # minus 2 length for first and final layers
        for i in range(len(self.nodes)-2):
            
            # NB: inputs are encoded as row vectors so no transposition is needed in these computations
            h_inputs = numpy.dot(layer_outputs[-1], self.weights[i]) + self.biases[i]
            layer_inputs.append(h_inputs.copy())
            h_outputs = self.hidden_activation(h_inputs)
            layer_outputs.append(h_outputs.copy())
        
        # final layer corresponds to last element in all lists and has a different activation, so left out of for loop above
        final_inputs = numpy.dot(layer_outputs[-1], self.weights[-1]) + self.biases[-1] 
        final_outputs = self.output_activation(final_inputs)

        # cross-entropy loss: sums output of -log of output activation for the output node corresponding to the correct label for that image
        # approaches 0 as activation for correct output nodes approaches 1
        # also guarantees that activation of other output nodes will be minimised since softmax returns probabilities that sum to 1
        loss = -numpy.sum(targets * numpy.log(final_outputs)) / (self.nodes[-1] * batch_size)

        # compiling a list of the gradient of each hidden layer outputs with respect to the layer inputs, for backpropagation later
        # this is simply the derivative of the activation function applied to the layer inputs
        self.hidden_derivs = []
        for i in range(len(self.nodes)-2):
            deriv_layer_output_input = deriv(self.hidden_fn, layer_inputs[i+1])
            self.hidden_derivs.append(deriv_layer_output_input)

        # linear factor from the loss calculation due to averaging over batches and output nodes, for loss gradient
        factor = 1 / (self.nodes[-1] * batch_size)
        
        # initialising a list of the gradient of the loss with respect to each layer's inputs
        # the first item is the gradient with respect to the final layer's inputs
        # i.e., derivative of the cross-entropy loss function applied to final outputs, times derivative of softmax applied to final inputs 
        # this simplifies to final outputs minus targets
        # due to the logarithmic and exponential nature of the softmax and cross-entropy functions respectively
        loss_derivs = [factor * (final_outputs - targets)]

        # computing the rest of the loss gradients (w.r.t. layer inputs)
        for i in range(len(self.nodes)-2):
            
            # gradients are computed backwards through the DNN so each time the weights and gradient from the next layer upstream is used
            # via chain rule, the gradient for each layer is the loss gradient of the previous (downstream) layer
            # multiplied (dotted) with the transpose of the weights between those layers
            # and then multiplied elementwise with the partial derivative of the hidden activation function for the (upstream) layer
            deriv_loss_layer_input = numpy.dot(loss_derivs[0], self.weights[-1-i].T) * self.hidden_derivs[-1-i]

            # adding backwards so list will be ordered ascendingly (from first to final layer), for ease of subsequent usage
            loss_derivs.insert(0, deriv_loss_layer_input)

        # finally the gradient of the loss w.r.t. the weights between each layer is
        # the partial derivative of the loss w.r.t. the downstream layer inputs multiplied (dotted) with
        # the transpose of the partial derivative of the downstream layer inputs w.r.t. the weights, which is the upstream layer outputs

        # updating all the weights by subtracting the relevant gradient (calculated as described above) times the LR
        for i in range(len(self.weights)):
            self.weights[i] += self.lr * -numpy.dot(layer_outputs[i].T, loss_derivs[i])

        # updates for biases are similar but averaged over the batch
        # also the partial derivative of the the downstream layer inputs w.r.t. the biases is simply the identity matrix
        # NB: multiplying by batch_size to 'undo' division by batch_size in the loss_derivs, since numpy.mean does this again
        for i in range(len(self.biases)):
            self.biases[i] += (self.lr * -numpy.mean(loss_derivs[i] * batch_size, axis=0))

        # loss monitoring updates
        # if first training run, set the loss tracker to equal the current loss
        if self.loss_tracker == 0:
            self.loss_tracker += loss
            
        # for future training runs update the loss using an exponential moving average (EMA) formula
        # the EMA is mostly the previous average with a small adjustment for the latest loss value
        # over multiple EMA updates the original average decays exponentially (hence the name)
        else:
            self.loss_tracker = self.alpha * self.loss_tracker + (1-self.alpha) * loss
        
        pass

    
    # defining a function to query the DNN, i.e. compute the DNN outputs for a given input
    # this function is generalised to returns inputs and outputs for all layers
    # it can be indexed in-line to select between these two and further indexed to select particular layers or even nodes
    # all of this functionality is required for visualisation
    def query(self, inputs_list):
        
        # ensuring inputs are of the right dimension and shape (corresponding to the input layer)
        inputs = numpy.reshape(numpy.array(inputs_list, ndmin=2), [1, self.nodes[0]])

        # initialising lists for layer inputs and outputs, as in the training function
        layer_inputs = [inputs]
        layer_outputs = [inputs]

        # computing the inputs and outputs from each hidden layer (by iteration), and adding them to the lists
        for i in range(len(self.nodes)-2):
            h_inputs = numpy.dot(layer_outputs[-1], self.weights[i]) + self.biases[i]
            layer_inputs.append(h_inputs.copy())
            h_outputs = self.hidden_activation(h_inputs)
            layer_outputs.append(h_outputs.copy())
        
        # computing inputs and outputs of final layer, and adding to lists
        final_inputs = numpy.dot(layer_outputs[-1], self.weights[-1]) + self.biases[-1]
        layer_inputs.append(final_inputs.copy())
        final_outputs = self.output_activation(final_inputs)
        layer_outputs.append(final_outputs.copy())

        # returns a list containing two lists, which themselves each contain arrays corresponding to each layer
        return [layer_outputs, layer_inputs]

In [ ]:
# choosing hyperparameters, such as no. of nodes in each layer
input_nodes = 784
output_nodes = 10

# hidden_nodes is a list to easily change number or size of layers
hidden_nodes = [1250, 1000, 750, 500, 250, 100, 25]

# initial learning rate
learning_rate = 0.1

# creating an instance of the DNN class with these hyperparameters
c = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

**RE-LOADING WEIGHTS AND BIASES** (OPTIONAL)

In [ ]:
# iterating over the number of layers and reloading weights and biases from saved files
# NB: this only works if the DNN c (as defined above) has the same number of layers and nodes as the saved DNN

for i in range(len(c.nodes)-1):
    c.weights[i] = numpy.loadtxt("/Users/benauer/Documents/DNN Saved/c_weights" + str(i) + "_saved.csv", delimiter=',')
    c.biases[i] = numpy.loadtxt("/Users/benauer/Documents/DNN Saved/c_biases" + str(i) + "_saved.csv", delimiter=',')

**CLASSIFIER TRAINING**

In [ ]:
# loading the mnist training data CSV file into a list
# note: the file path below should of course be replaced for a new user
training_data_file = open("/Users/benauer/Downloads/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

**NB:** The following two cells (until Performance Testing) can be skipped if re-loading a saved DNN.

In [ ]:
# restating alpha and LR definitions here for ease of finetuning between training epochs (e.g. manual LR decay)
c.alpha = 0.9999
c.lr = 0.1

In [ ]:
# training the neural network in batches for greater efficiency and speed
batch_size = 32

# epochs is the number of times that training loops through the entire MNIST training data set
epochs = 10

for e in range(epochs):

    # shuffles order of data set for each epoch
    numpy.random.shuffle(training_data_list)

    # in each epoch, training iterates over the data set in intervals equal to the batch size, with inputs and targets compiled for that batch
    for i in range(0, len(training_data_list), batch_size):
        batch = training_data_list[i:i+batch_size]

        batch_inputs = []
        batch_targets = []
        
        for record in batch:
            # extracting numerical pixel values
            all_values = record.split(',')
            # scaling the inputs
            # value range [0,1] is optimal for ReLU so no shifting required
            inputs = numpy.asfarray(all_values[1:]) / 255.0
            # encoding the target label as a one-hot vector with same value range [0,1]
            targets = numpy.zeros(output_nodes)
            # all_values[0] is the target label for this record
            targets[int(all_values[0])] = 1

            batch_inputs.append(inputs)
            batch_targets.append(targets)

        # train function automatically works via standard matrix operations with 2 dimensional inputs so no further processing is required
        c.train(batch_inputs, batch_targets)

        # loss monitoring
        # printing the epoch number and EMA loss
        # prints approximately 10 times per epoch
        # by checking if the training set size divided by 10, and rounded to the nearest multiple of the batch size, divides i
        if i % (batch_size * round(len(training_data_list) / 10 / batch_size)) == 0:
            print(e+1,":", c.loss_tracker)

        pass
    pass

**CLASSIFIER PERFORMANCE TESTING**

In [ ]:
# loading the mnist test data CSV file into a list
# again, the file path must be replaced for a new user
test_data_file = open("/Users/benauer/downloads/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [ ]:
# initialising scorecard for the classifier's test performance
scorecard = []

# iterating once through the test data set
for record in test_data_list:
    # extracting numerical pixel values
    all_values = record.split(',')
    # the first item of each record is the correct label (integer from 0 to 9 in this case)
    correct_label = int(all_values[0])
    # scaling the inputs
    # again, no shifting required
    inputs = numpy.asfarray(all_values[1:]) / 255.0
    # using query function to do a forward pass with each input
    # length of the node numbers list is 1 greater than the index of the final layer
    outputs = c.query(inputs)[0][len(c.nodes)-1]
    # taking the index of the output node with the highest activation, i.e. the DNN's highest probability classification, or 'best guess'
    label = numpy.argmax(outputs)

    if (label == correct_label):
        # DNN's best guess matches correct answer, so add 1 to scorecard
        scorecard.append(1)
    else:
        # DNN's best guess doesn't match correct answer, so add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [ ]:
# calculating the performance score, no. of correct answers divided by no. of total answers
scorecard_array = numpy.asarray(scorecard)
print ("performance =", scorecard_array.sum() / scorecard_array.size)

**SAVING WEIGHTS AND BIASES** (OPTIONAL)

In [ ]:
c_weights_saved = c.weights.copy()
c_biases_saved = c.biases.copy()

# saving the DNN weights and biases to files in case current session is lost
# each layer's weights and biases must be saved to a separate file since dimensions for different layers do not align

# note: the file paths below are simply examples and should be replaced for each user

for i in range(len(c.weights)):
    numpy.savetxt("/Users/benauer/Documents/DNN Saved/c_weights" + str(i) + "_saved.csv", c_weights_saved[i], delimiter=",")
    numpy.savetxt("/Users/benauer/Documents/DNN Saved/c_biases" + str(i) + "_saved.csv", c_biases_saved[i], delimiter=",")

**VISUALISATION BY OPTIMISATION**

In [ ]:
# importing matplotlib.pyplot for visual depictions of images
import matplotlib
import matplotlib.pyplot

# defining the grayscale cmap for ease of reference in all pyplot calls later
gray = matplotlib.pyplot.get_cmap('gray')

# creating a combined list of the MNIST test and training data for initialisation and averages later
combined_full_data = training_data_list[:] + test_data_list[:]

# storing labels separately
combined_labels = []

for i in range(len(combined_full_data)):
    combined_labels.append(int(combined_full_data[i][0]))
    
    # extracting pixel values
    image_values = combined_full_data[i][2:-1].split(',')
    # replacing each entry in the dataset with an array of the image values excluding the label
    combined_full_data[i] = numpy.asfarray(image_values) / 255.0

In [ ]:
# parameters for numpy.clip function later (to restrict allowed pixel values for images)
cmin, cmax = 0, 1

# derivative of numpy.clip for backpropagation in image optimisation below
def clip_deriv(x, cmin, cmax):
    return numpy.where((x >= cmin) & (x <= cmax), 1, 0)

In [ ]:
def act_grad_layer(DNN, img, tlayer):
    '''
    This function computes the gradient of the activation of (all nodes in) a target layer in a specified classifier DNN with respect to an
    image input, by backpropagating the layer activation through the previous (upstream) layers of the DNN.
    It takes as input a DNN, an image in a [1,784] array format, and the number of the target layer (as an integer).
    It returns the gradient as another [1,784] array.
    '''

    # querying the DNN once and gathering the resulting data for maximum efficiency
    forward_pass = DNN.query(img)
    layer_inputs = forward_pass[1]
    layer_outputs = forward_pass[0]

    # firstly the partial derivatives of each layer's outputs with respect to it's inputs are computed
    # this is simply the derivative of the hidden activation function (e.g. ReLU) applied to the layer inputs
    hidden_derivs = []
    for i in range(len(DNN.nodes)-2):
        deriv_layer_output_input = deriv(DNN.hidden_fn, layer_inputs[i+1])
        hidden_derivs.append(deriv_layer_output_input)

    # next the partial derivatives of the tlayer activation w.r.t. the input to each layer are computed, moving backwards through the layers
    # the final partial derivative following this pattern will be for the activation w.r.t. the input image which is the desired output
    
    # if the target layer is a hidden layer then all the partial derivatives use only the hidden function
    if tlayer < len(DNN.nodes)-1:
        
        # the first partial derivative w.r.t. tlayer inputs is simply the hidden_derivs item for the target layer, since the activations are
        # the tlayer outputs (item tlayer-1 since there is no entry for layer 0 in the hidden_derivs list)
        
        activation_derivs = [hidden_derivs[tlayer-1]]

    # calculations will be done a similar way when targeting the output layer but with the output function for the first partial derivative
    elif tlayer == len(DNN.nodes)-1:

        # NB: the output function SoftMax depends on all inputs to the output layer for any individual output activation
        # hence the derivative returns a [10,10] matrix, where each row is the gradient of a particular output node's activation
        # the intention is to combine the effects of these derivatives anyway (from all the non-target nodes)
        # so the matrix can be summed along the rows here, which is equivalent to summing after backpropagation but more efficient
        
        activation_derivs = [numpy.sum(deriv(DNN.output_fn, layer_inputs[tlayer]), axis=0)]

    # if tlayer is the 1st hidden layer, there are no more hidden layer gradients to be computed (only gradient w.r.t. inputs)
    if tlayer != 1:

        # computing gradients for the rest of the hidden layers until the first hidden layer

        # the partial derivative w.r.t. the inputs of each layer tlayer-1-j is the first partial derivative from above (w.r.t. tlayer)
        # multiplied (dotted) with the transpose of the tlayer-1-j weights and multiplied elementwise with the
        # partial derivative of the outputs of tlayer-1-j w.r.t. its inputs (item tlayer-2 in hidden_derivs list)
        for j in range(tlayer-1):
            deriv_act_layer_input = numpy.dot(activation_derivs[0], DNN.weights[tlayer-1-j].T) * hidden_derivs[tlayer-2-j]
            
            activation_derivs.insert(0, deriv_act_layer_input)

    # gradient w.r.t. the first hidden layer inputs is now element 0 of activation_derivs
    # to get the gradient w.r.t. the image this simply needs to be multiplied (dotted) with the weights between layer 0 and 1
    # since layer 0 (the image values) has no activation function
    act_deriv_image = numpy.dot(activation_derivs[0], DNN.weights[0].T)

    return act_deriv_image

In [ ]:
def act_grad_node(DNN, img, tlayer, tnode):
    '''
    This function is very similar to act_grad_layer but it computes the gradient of the activation of a particular node in a target layer
    w.r.t. the image input, as opposed to the gradient of the activations of the entire layer.

    Again, it takes a DNN, an image array, and a target layer number as input, but also the target node number.
    It returns the gradient as another [1,784] array.

    Both functions are used below to optimise an image for exclusive activation of a target node in a layer (for visualisation).
    '''

    # gathering DNN query data
    forward_pass = DNN.query(img)
    layer_inputs = forward_pass[1]
    layer_outputs = forward_pass[0]

    # computing partial derivatives of each layers outputs w.r.t. inputs again
    hidden_derivs = []
    for i in range(len(DNN.nodes)-2):
        deriv_layer_output_input = deriv(DNN.hidden_fn, layer_inputs[i+1])
        hidden_derivs.append(deriv_layer_output_input)

    # the 'layer inputs' to the ith node (a scalar) depends on the ith column only of the weights from the previous layer
    DNN_weights_to_node = DNN.weights[tlayer-1][:, [tnode]]
    inputs_to_node = numpy.dot(layer_outputs[tlayer-1], DNN_weights_to_node) + DNN.biases[tlayer-1][tnode]

    # backpropagating through DNN to compute partial derivatives of tnode activation w.r.t. the inputs to each layer
    # separating hidden and output target layers again for the same reason
    if tlayer < len(DNN.nodes)-1:
        
        # the first partial derivative w.r.t. tnode inputs is calculated from inputs_to_node so must be separate from hidden_derivs
        # list which is based on entire layers
        activation_derivs = [deriv(DNN.hidden_fn, inputs_to_node)]

        # multiplying first partial derivative w.r.t. tlayer-1 inputs
        # when looking at the tnode specifically this part of the gradient is different for the hidden and output layers
        # so this item cannot be incorporated into the general 'for loop' below
        # this step should also be skipped if the target layer is the 1st hidden layer, as in the previous function
        if tlayer != 1:
            activation_derivs.insert(0, numpy.dot(activation_derivs[0], DNN_weights_to_node.T) * hidden_derivs[tlayer-1-1])

    # recall the SoftMax derivative returns a [10,10] matrix
    # the relevant row of this matrix ([1,10]) must be indexed for the partial derivative of the target node activation
    elif tlayer == len(DNN.nodes)-1:
        activation_derivs = [deriv(DNN.output_fn, layer_inputs[tlayer])[tnode, :]]

        # first partial derivative from above is already [1,10], so it is multiplied by the entire weights from tlayer-1 (transposed)
        # as opposed to just the weights to that node (when targeting hidden layers)
        activation_derivs.insert(0, numpy.dot(activation_derivs[0], DNN.weights[tlayer-1].T) * hidden_derivs[tlayer-1-1])

    # again, skipping the for loop if tlayer is the 1st hidden layer
    if tlayer != 1:

        # computing gradients for the rest of the hidden layers
        for j in range(tlayer-2):
            deriv_act_layer_input = numpy.dot(activation_derivs[0], DNN.weights[tlayer-2-j].T) * hidden_derivs[tlayer-3-j]
            
            activation_derivs.insert(0, deriv_act_layer_input)

        # again, gradient w.r.t. the image is now simply element 0 of activation_derivs times the weights for layer 0
        act_deriv_image = numpy.dot(activation_derivs[0], DNN.weights[0].T)

    # if tlayer is the 1st hidden layer, the partial derivative w.r.t. to the layer inputs is a scalar ([1,1]) so must be multiplied (dotted)
    # with the row vector (transposed) DNN_weights_to_node.T, which is [1,784]
    elif tlayer == 1:
        act_deriv_image = numpy.dot(activation_derivs[0], DNN_weights_to_node.T)

    return act_deriv_image

In [ ]:
# defining a new class for visualising the nodes of a classifier DNN
class visualisation:

    # the image is first initialised, with all self parameters declared
    def __init__(self, initialisation):

        # there are three options for the choice of initialisation, two of which are specified as strings
        if type(initialisation) == str:
            # 'zero' starts at near-zero values for all pixels (not completely zero to avoid any vanishing gradient problems)
            if initialisation == 'zero':
                self.values = numpy.zeros([1,784]) + 0.01
            # 'random' samples pixel values from a normal distribution with mean 0.5 and standard deviation 0.1
            # this keeps pixels more or less entirely within the range 0 to 1
            elif initialisation == 'random':
                self.values = numpy.random.normal(0.5, 0.1, [1,784])

        # the third option simply initialises from a specified image, which is reshaped if necessary
        else:
            self.values = numpy.reshape(initialisation.copy(), [1,784])

        # defining variables for monitoring training
        
        # these will be updated with the latest activation values for the target node and other nodes respectively
        # using an exponential moving average (EMA) formula (see below)
        
        # these measures are stored and displayed separately for more fine-grained information about the image during optimisation
        # as opposed to in a combined 'loss function'
        self.t_activation_ema = 0
        self.o_activation_ema = 0
        self.tvreg_ema = 0
        self.l1reg_ema = 0
        self.second_max_ema = 0

        self.loss_ema = 0

        # smoothing parameter for the EMA formula (default value here)
        self.alpha = 0.9

        pass

    def train(self, DNN, tlayer, tnode, learningrate, penaltyO, TVweight, L1weight):
        '''
        This function optimises / 'trains' the image for activation of a target node in a target layer of a specified DNN, by backpropagation
        using the act_grad_node function.
        
        It takes as input a DNN, the numbers of the target layer and node, and several parameters relating to the optimisation process itself,
        specifically:
        learningrate: The learning rate
        penaltyO: The weighting applied to penalise activations of non-target (Other) nodes in the target layer
        TVweight: The weighting applied to TV regularisation during optimisation
        L1weight: The weighting applied to L1 regularisation during optimisation

        Note that a numpy.clip 'activation function' is applied to the image at the start of each optimisation step, and this is used to
        measure activations and all other regularisation quantities.
        
        The function then updates the (raw / non-activated) values of the image (self.values) based on the gradients from act_grad_node and
        derivatives of the other functions used (for activation and regularisation), before being called again for the next optimisation step.
        '''

        # firstly all metrics for the current image are calculated
        
        # limiting pixel values in the image to the range specified above
        # normally 0 to 1 since this is the same range as the normalised MNIST images used to train and test the classifier
        clip_values = numpy.clip(self.values, cmin, cmax)
        
        # gathering data from forward pass immediately for efficiency
        # only layer outputs needed since act_grad_node function handles layer inputs
        layer_outputs = DNN.query(clip_values)[0]

        # extracting target node activation, the primary indicator of optimisation success
        tnode_activation = layer_outputs[tlayer][0][tnode]

        # taking the mean of the activations of all other nodes (hence tnode_activation is subtracted from the sum)
        # the layer outputs are from ReLU or sigmoid so there is no risk of negative and positive values cancelling each other out
        other_activations = (numpy.sum(layer_outputs[tlayer]) - tnode_activation) / (DNN.nodes[tlayer] - 1)

        # the activation of the second-most activated node after the target node can be used instead of the mean activation of other nodes
        # as an indicator of the success of optimisation for the target node (relative to other nodes)
        if numpy.argmax(layer_outputs[tlayer][0]) == tnode:
            second_max_idx = numpy.argsort(layer_outputs[tlayer][0])[-2]
            second_max = layer_outputs[tlayer][0][second_max_idx]

        # if tnode does not have the maximum activation then the 'second_max' is actually just the first maximum
        else:
            second_max_idx = numpy.argmax(layer_outputs[tlayer][0])
            second_max = layer_outputs[tlayer][0][second_max_idx]

        # L1 regularisation measures the sum of the absolute values of all pixels to discourage unnecessary and/or extreme pixel values
        l1_regularisation = numpy.sum(numpy.abs(clip_values))
        
        # TV regularisation measures the difference between versions of the image translated 1 pixel each way on both the vertical and
        # horizontal axes, which is most easily done with the image in a [28,28] format
        image = numpy.reshape(clip_values, [28,28])
        tv_regularisation = numpy.sum(numpy.abs(image[1:,:] - image[:-1,:])) + numpy.sum(numpy.abs(image[:,1:] - image[:,:-1]))

        # the loss corresponds to the success of optimisation w.r.t. all measures in combination
        # this includes the regularisation measures, weighted as in the gradients below
        # the loss does not affect optimisation itself but can be an informative metric to monitor
        loss = -(tnode_activation - penaltyO * other_activations - TVweight * tv_regularisation - L1weight * l1_regularisation)
        
        # next, backpropagation is conducted

        # tgrad is the gradient of the target node activation w.r.t. the original unclipped image values
        # this is the tnode activation gradient w.r.t. the clipped image times the derivative of numpy.clip applied to the original values
        tgrad = act_grad_node(DNN, clip_values, tlayer, tnode) * clip_deriv(self.values, cmin, cmax)

        # ograd is used to optimise against activation of other nodes in the target layer
        ograd = (act_grad_layer(DNN, clip_values, tlayer) * clip_deriv(self.values, cmin, cmax) - tgrad) / (DNN.nodes[tlayer] - 1)

        # NB: clip_deriv is deliberately not included in the L1 and TV gradients to prevent clip from 'killing' certain pixels permanently
        # i.e. clip_deriv would normally cause the gradients of clipped pixels (outside the bounds) to become 0 thereafter
        # which would 'trap' these pixels at the bounds and create very sharp images, appearing unregularised
        
        # but with this method TV or L1 regularisation can keep these pixels dynamic, when that is beneficial for regularisation
        # also note clip_deriv is simply 1 within the bounds so there are no other effects of removing it

        # l1_grad is the gradient of the L1 quantity w.r.t. the clipped values
        l1_grad = numpy.sign(clip_values)

        # tv_grad is the gradient of the TV quantity w.r.t. the clipped values
        # the image is padded by 1 pixel first to enable efficient comparison of neighbouring pixels for tv_grad (in one line below)
        padded = numpy.pad(image, pad_width=1)
        tv_grad = (numpy.sign(image - padded[:-2, 1:-1]) + numpy.sign(image - padded[2:, 1:-1]) + numpy.sign(image - padded[1:-1, :-2]) + numpy.sign(image - padded[1:-1, 2:]))
        
        # again, TV computations are done with a [28,28] image so this needs to be reshaped before incorporation with the other gradients
        tv_grad_reshaped = numpy.reshape(tv_grad, [1,784])

        # the unclipped image values are then updated based on the learning rate, and all the gradients, and their respective weightings
        # this is essentially minus the gradient of the loss w.r.t. the image times the learning rate
        self.values += learningrate * (tgrad - penaltyO * ograd - TVweight * tv_grad_reshaped - L1weight * l1_grad)

        # finally, the relevant training quantities (and the loss) are monitored

        # on the first optimisation step, each quantity is simply set to its current value to avoid starting the EMA from 0
        if self.t_activation_ema == 0:
            self.t_activation_ema += tnode_activation
            self.o_activation_ema += other_activations
            self.tvreg_ema += tv_regularisation
            self.l1reg_ema += l1_regularisation
            self.second_max_ema += second_max
            
            self.loss_ema += loss

        # in future steps, the EMAs are updated according to the EMA formula with the same alpha value
        # as in DNN training, the alpha value determines the degree of average smoothing
        # i.e., how much the EMA is influenced by the new value for each quantity
        else:
            self.t_activation_ema = self.alpha * self.t_activation_ema + (1-self.alpha) * tnode_activation
            self.o_activation_ema = self.alpha * self.o_activation_ema + (1-self.alpha) * other_activations
            self.tvreg_ema = self.alpha * self.tvreg_ema + (1-self.alpha) * tv_regularisation
            self.l1reg_ema = self.alpha * self.l1reg_ema + (1-self.alpha) * l1_regularisation
            self.second_max_ema = self.alpha * self.second_max_ema + (1-self.alpha) * second_max
            
            self.loss_ema = self.alpha * self.loss_ema + (1-self.alpha) * loss
        
        pass

In [ ]:
# initialising an image for optimisation
# near-zero values is the default, but random values can also be used or an MNIST image from combined_full_data
v = visualisation('zero')

# setting targets for optimisation
DNN = c
target_layer = 7
target_node = 1

In [ ]:
# restating hyper-parameters for ease of fine-tuning
# these will be input into the train function below
learningrate = 0.1
TVweight = 0.256
L1weight = 0.0256
# penalty term for other activations is usually kept at 0 but can be adjusted for experimental purposes
penaltyO = 0

# fine-tuning alpha for the EMA formulas
# a low value such as 0.9, or even 0, is normally sufficient, since optimisation is quite rapid unless the LR is very small
v.alpha = 0.9

In [ ]:
# steps is the number of times that the train function will be called and the image values will be updated
steps = 30

# note: it is also straightforward to nest this training loop within other loops to optimise many images consecutively

for s in range(steps):

    # train is called using the values defined in the previous cell
    v.train(c, target_layer, target_node, learningrate, penaltyO, TVweight, L1weight)

    # every 10 steps, the current step number is printed next to (the EMA of) all the tracked quantities
    # these are also printed at the start and end of optimisation
    if (s % 10 == 0) or (s == 0) or (s == steps-1):
        
        # the first line below displays the mean activation of other nodes in tlayer
        # while the second line (the default) displays only the highest other activation
        # these can be selected between
        
        #print(str(s),":", f"{v.t_activation_ema:.6f}", f"{v.o_activation_ema:.6f}", f"{v.tvreg_ema:.6f}", f"{v.l1reg_ema:.6f}", f"{v.loss_ema:.6f}")
        print(str(s),":", f"{v.t_activation_ema:.6f}", f"{v.second_max_ema:.6f}", f"{v.tvreg_ema:.6f}", f"{v.l1reg_ema:.6f}", f"{v.loss_ema:.6f}")

    # if 50 steps are completed without the target node activation increasing beyond 0
    # the node is most likely 'dead', meaning it does not respond to any input images
    # this threshold can be manually adjusted here of course
    if s == 50:
        if v.t_activation_ema == 0:
            print("Dead node.")
            break
    
    pass

In [ ]:
# reshaping the values into a [28,28] image format
vif = numpy.reshape(numpy.clip(v.values,0,1), [28,28])

# displaying the image
matplotlib.pyplot.imshow(vif, cmap = gray)

In [ ]:
# querying the network for the activations induced by the optimised image in the target layer and the output layer

# temporarily setting numpy to print all array values rather than an abbreviated version with ellipses
with numpy.printoptions(threshold=numpy.inf):
    print(c.query(numpy.clip(v.values,0,1))[0][target_layer][0])

    # reshaping output activations to display them horizontally
    print(numpy.reshape(c.query(numpy.clip(v.values,0,1))[0][-1][0], [10,1]))

**SAVING IMAGES** (OPTIONAL)

In [ ]:
# storing the (reshaped) image values, target layer activations and output activations
f_img = numpy.reshape(numpy.clip(v.values, 0, 1), [28,28])
f_tlayer_activations = c.query(numpy.clip(v.values, 0, 1))[0][target_layer][0]
f_output_activations = c.query(numpy.clip(v.values, 0, 1))[0][-1][0]

# note: the following file paths are simply examples and should be replaced
# of course the final values from training can also be referenced in the file names

fimg_file = "/Users/benauer/Documents/Example Folder/Example Image.png"
fvalues_file = "/Users/benauer/Documents/Example Folder/Example Image Values.csv"
factivations_file = "/Users/benauer/Documents/Example Folder/Example TLayer Activations.csv"
foutput_file = "/Users/benauer/Documents/Example Folder/Example Output Activations.csv"

In [ ]:
# defining the settings for the saved image
# these settings save the image at a desired size without any distortion or modifications

# the figure is initially given a size in inches with no frame
# by default the size is 1x1 inches but can be adjusted
# this is later multiplied by the DPI 28 so in_size determines the eventual size in pixels as a multiple of 28x28
in_size = 1
fig = matplotlib.pyplot.figure(figsize=(in_size,in_size), frameon=False)

# ensuring the image fills the entire space in the saved image (including by disabling the display of axes)
ax = matplotlib.pyplot.Axes(fig, [0, 0, 1, 1])
ax.set_axis_off()
fig.add_axes(ax)

# defining the image as monochrome gray with no interpolation to prevent blurring of pixels if scaling up
ax.imshow(f_img, cmap = gray, interpolation='none')

# saving the open figure fig, with DPI 28, meaning the final dimensions are DPI * in_size pixels
matplotlib.pyplot.savefig(fimg_file, dpi=28, pad_inches=0)

In [ ]:
# saving the values to their respective files
# this is very straightforward with numpy using savetxt (and loadtxt later)
# activations are stored to 10 decimal places while pixel values are stored with maximum precision
# all values are separated by commas

numpy.savetxt(fvalues_file, numpy.clip(v.values,0,1), delimiter=",")
numpy.savetxt(factivations_file, f_tlayer_activations, fmt='%.10f', delimiter=",")
numpy.savetxt(foutput_file, f_output_activations, fmt='%.10f', delimiter=",")

**LOADING IMAGES** (OPTIONAL)

In [ ]:
# example_file should be replaced with any CSV containing the full array values of the image to be loaded
example_file = "/Users/benauer/Documents/TV + L1 Examples/Layer 7; TV 0.25; L1 0.025!/1; 30 steps; 21.524354 act.csv"

# a numpy array is then defined with these values and displayed to verify whether the image has been loaded correctly
loaded_image = numpy.loadtxt(example_file, delimiter=',')
loaded_reshaped = numpy.reshape(loaded_image, [28,28])
matplotlib.pyplot.imshow(loaded_reshaped, cmap = gray)

# once loaded the image can be re-used for various purposes, e.g. to query the DNN again or for further optimisation

**AVERAGE IMAGE GENERATOR**

In [ ]:
# initialising a list of 1x784 zero vectors corresponding to each digit
avg_imgs = [numpy.zeros([1,784])] * 9

# NB: this loop can take many minutes to run to completion

for onode in range(c.nodes[-1]):
    amount = 0

    # looping through combined_full_data for each output node and adding relevant image values to that vector
    for i in range(len(combined_full_data)):

        # the threshold for adding images is 0.9 activation by default but can be decreased or increased
        if c.query(combined_full_data[i])[0][-1][0][onode] > 0.9:
            avg_imgs[onode] += combined_full_data[i]
            amount += 1

        # print statements report progress through each digit as a percentage
        if i % (len(combined_full_data) / 100) == 0:
            print(str(onode), ": ", f"{100 * i / len(combined_full_data):.0f}", "%")

    # average image is divided by the number of images that were added to normalise values
    avg_imgs[onode] = avg_imgs[onode] / amount